In [1]:
from enum import Enum, unique
import toml
import numpy as np
from rich.tree import Tree
import pandas as pd
from tqdm import tqdm

import sys
sys.path.append('../src')
from sysloss.components import *
from sysloss.components import ComponentTypes, _get_opt, _get_mand, _get_eff, RS_DEFAULT, LIMITS_DEFAULT

In [18]:
import rustworkx as rx
import numpy as np
from rich.tree import Tree
import json
import pandas as pd

class System:
    """System to be analyzed."""

    def __init__(self, name: str, source):
        self._g = None
        if not isinstance(source, Source):
            raise ValueError("Error: First component of system must be a source!")

        self._g = rx.PyDAG(check_cycle=True, multigraph=False, attrs={})
        pidx = self._g.add_node(source)
        self._g.attrs["name"] = name
        self._g.attrs["nodes"] = {}
        self._g.attrs["nodes"][source.params["name"]] = pidx

    @classmethod
    def from_file(cls, fname: str):
        """Load system from json file"""
        with open(fname, "r") as f:
            sys = json.load(f)

        entires = list(sys.keys())
        sysname = _get_mand(sys, "name")
        # add sources
        for e in range(1, len(entires)):
            vo = _get_mand(sys[entires[e]]["params"], "vo")
            rs = _get_opt(sys[entires[e]]["params"], "rs", RS_DEFAULT)
            lim = _get_opt(sys[entires[e]], "limits", LIMITS_DEFAULT)
            if e == 1:
                self = cls(sysname, Source(entires[e], vo=vo, rs=rs, limits=lim))
            else:
                self.add_source(Source(entires[e], vo=vo, rs=rs, limits=lim))
            # add childs
            if sys[entires[e]]["childs"] != {}:
                for p in list(sys[entires[e]]["childs"].keys()):
                    for c in sys[entires[e]]["childs"][p]:
                        cname = _get_mand(c["params"], "name")
                        # print("  " + cname)
                        limits = _get_opt(c, "limits", LIMITS_DEFAULT)
                        iq = _get_opt(c["params"], "iq", 0.0)
                        if c["type"] == "CONVERTER":
                            vo = _get_mand(c["params"], "vo")
                            eff = _get_mand(c["params"], "eff")
                            self.add_comp(
                                p,
                                comp=Converter(
                                    cname, vo=vo, eff=eff, iq=iq, limits=limits
                                ),
                            )
                        elif c["type"] == "LINREG":
                            vo = _get_mand(c["params"], "vo")
                            vdrop = _get_opt(c["params"], "vdrop", 0.0)
                            self.add_comp(
                                p,
                                comp=LinReg(
                                    cname, vo=vo, vdrop=vdrop, iq=iq, limits=limits
                                ),
                            )
                        elif c["type"] == "LOSS":
                            rs = _get_mand(c["params"], "rs")
                            vdrop = _get_mand(c["params"], "vdrop")
                            self.add_comp(
                                p, comp=Loss(cname, rs=rs, vdrop=vdrop, limits=limits)
                            )
                        elif c["type"] == "LOAD":
                            if "pwr" in c["params"]:
                                pwr = _get_mand(c["params"], "pwr")
                                self.add_comp(
                                    p, comp=PLoad(cname, pwr=pwr, limits=limits)
                                )
                            elif "rs" in c["params"]:
                                rs = _get_mand(c["params"], "rs")
                                self.add_comp(
                                    p, comp=RLoad(cname, rs=rs, limits=limits)
                                )
                            else:
                                ii = _get_mand(c["params"], "ii")
                                self.add_comp(
                                    p, comp=ILoad(cname, ii=ii, limits=limits)
                                )

        return self

    def __get_index(self, name: str):
        """Get node index from component name"""
        if name in self._g.attrs["nodes"]:
            return self._g.attrs["nodes"][name]

        return -1

    def __chk_parent(self, parent: str):
        """Check if parent exists"""
        if not parent in self._g.attrs["nodes"].keys():
            raise ValueError('Error: Parent name "{}" not found!'.format(parent))

        return True

    def __chk_name(self, name: str):
        """Check if component name is valid"""
        # check if name exists
        if name in self._g.attrs["nodes"].keys():
            raise ValueError('Error: Component name "{}" is already used!'.format(name))

        return True

    def __get_childs_tree(self, node):
        """Get dict of parent/childs"""
        childs = list(rx.bfs_successors(self._g, node))
        cdict = {}
        for c in childs:
            cs = []
            for l in c[1]:
                cs += [self._g.attrs["nodes"][l.params["name"]]]
            cdict[self._g.attrs["nodes"][c[0].params["name"]]] = cs
        return cdict

    def __get_nodes(self):
        """Get list of nodes in system"""
        return [n for n in self._g.node_indices()]

    def __get_childs(self):
        """Get list of children of each node"""
        nodes = self.__get_nodes()
        cs = list(-np.ones(max(nodes) + 1, dtype=np.int32))
        for n in nodes:
            if self._g.out_degree(n) > 0:
                ind = [i for i in self._g.successor_indices(n)]
                cs[n] = ind
        return cs

    def __get_parents(self):
        """Get list of parent of each node"""
        nodes = self.__get_nodes()
        ps = list(-np.ones(max(nodes) + 1, dtype=np.int32))
        for n in nodes:
            if self._g.in_degree(n) > 0:
                ind = [i for i in self._g.predecessor_indices(n)]
                ps[n] = ind
        return ps

    def __get_sources(self):
        """Get list of sources"""
        tn = [n for n in rx.topological_sort(self._g)]
        return [n for n in tn if isinstance(self._g[n], Source)]

    def __get_topo_sort(self):
        """Get nodes topological sorted"""
        tps = rx.topological_sort(self._g)
        return [n for n in tps]

    def __sys_vars(self):
        """Get system variable lists"""
        vn = max(self.__get_nodes()) + 1  # highest node index + 1
        v = list(np.zeros(vn))  # voltages
        i = list(np.zeros(vn))  # currents
        return v, i

    def __make_rtree(self, adj, node):
        """Create Rich tree"""
        tree = Tree(node)
        for child in adj.get(node, []):
            tree.add(self.__make_rtree(adj, child))
        return tree

    def add_comp(self, parent: str, *, comp):
        """Add component to system"""
        # check that parent exists
        self.__chk_parent(parent)
        # check that component name is unique
        self.__chk_name(comp.params["name"])
        # check that parent allows component type as child
        pidx = self.__get_index(parent)
        if not comp.component_type in self._g[pidx].child_types:
            raise ValueError(
                "Error: Parent does not allow child of type {}!".format(
                    comp.component_type.name
                )
            )
        cidx = self._g.add_child(pidx, comp, None)
        self._g.attrs["nodes"][comp.params["name"]] = cidx

    def add_source(self, source):
        """Add new source"""
        self.__chk_name(source.params["name"])
        if not isinstance(source, Source):
            raise ValueError("Error: Component must be a source!")

        pidx = self._g.add_node(source)
        self._g.attrs["nodes"][source.params["name"]] = pidx

    def change_comp(self, name: str, *, comp):
        """Replace component with a new one"""
        # if component name changes, check that it is unique
        if name != comp.params["name"]:
            self.__chk_name(comp.params["name"])

        # check that parent allows component type as child
        eidx = self.__get_index(name)
        parents = self.__get_parents()
        if parents[eidx] != -1:
            if not comp.component_type in self._g[parents[eidx][0]].child_types:
                raise ValueError(
                    "Error: Parent does not allow child of type {}!".format(
                        comp.component_type.name
                    )
                )
        self._g[eidx] = comp
        # replace node name in graph dict
        del [self._g.attrs["nodes"][name]]
        self._g.attrs["nodes"][comp.params["name"]] = eidx

    def del_comp(self, name: str, *, del_childs: bool = True):
        eidx = self.__get_index(name)
        if eidx == -1:
            raise ValueError("Error: Component name does not exist!")
        parents = self.__get_parents()
        if parents[eidx] == -1:
            raise ValueError("Error: Cannot delete source node!")
        childs = self.__get_childs()
        # if not leaf, check if child type is allowed by parent type (not possible?)
        # if leaves[eidx] == 0:
        #     for c in childs[eidx]:
        #         if not self._g[c].component_type in self._g[parents[eidx]].child_types:
        #             raise ValueError(
        #                 "Error: Parent and child of component are not compatible!"
        #             )
        # delete childs first if selected
        if del_childs:
            for c in rx.descendants(self._g, eidx):
                print(c, eidx)
                del [self._g.attrs["nodes"][self._g[c].params["name"]]]
                self._g.remove_node(c)
        # delete node
        self._g.remove_node(eidx)
        del [self._g.attrs["nodes"][name]]
        # restore links between new parent and childs, unless deleted
        if not del_childs:
            if childs[eidx] != -1:
                for c in childs[eidx]:
                    self._g.add_edge(parents[eidx][0], c, None)

    def tree(self, name=""):
        """Print tree structure, starting from node name"""
        if not name == "":
            if not name in self._g.attrs["nodes"].keys():
                raise ValueError("Error: Component name is not valid!")
            root = [name]
        else:
            ridx = self.__get_sources()
            root = [self._g[n].params["name"] for n in ridx]

        t = Tree(self._g.attrs["name"])
        for n in root:
            adj = rx.bfs_successors(self._g, self._g.attrs["nodes"][n])
            ndict = {}
            for i in adj:
                c = []
                for j in i[1]:
                    c += [j.params["name"]]
                ndict[i[0].params["name"]] = c
            t.add(self.__make_rtree(ndict, n))
        return t

    def __sys_init(self):
        """Create vectors of init values for solver"""
        v, i = self.__sys_vars()
        for n in self.__get_nodes():
            v[n] = self._g[n]._get_outp_voltage()
            i[n] = self._g[n]._get_inp_current()
        return v, i

    def __fwd_prop(self, v: float, i: float):
        """Forward propagation of voltages"""
        vo, _ = self.__sys_vars()
        # update output voltages (per node)
        for n in self._topo_nodes:
            p = self._parents[n]
            if self._childs[n] == -1:  # leaf
                if p == -1:  # root
                    vo[n] = self._g[n]._solv_outp_volt(0.0, 0.0, 0.0)
                else:
                    vo[n] = self._g[n]._solv_outp_volt(v[p[0]], i[n], 0.0)
            else:
                # add currents into childs
                isum = 0
                for c in self._childs[n]:
                    isum += i[c]
                if p == -1:  # root
                    vo[n] = self._g[n]._solv_outp_volt(0.0, 0.0, isum)
                else:
                    vo[n] = self._g[n]._solv_outp_volt(v[p[0]], i[n], isum)
        return vo

    def __back_prop(self, v: float, i: float):
        """Backward propagation of currents"""
        _, ii = self.__sys_vars()
        # update input currents (per node)
        for n in self._topo_nodes[::-1]:
            p = self._parents[n]
            if self._childs[n] == -1:  # leaf
                if p == -1:  # root
                    ii[n] = self._g[n]._solv_inp_curr(v[n], 0.0, 0.0)
                else:
                    ii[n] = self._g[n]._solv_inp_curr(v[p[0]], 0.0, 0.0)
            else:
                isum = 0.0
                for c in self._childs[n]:
                    isum += i[c]
                if p == -1:  # root
                    ii[n] = self._g[n]._solv_inp_curr(v[n], v[n], isum)
                else:
                    ii[n] = self._g[n]._solv_inp_curr(v[p[0]], v[n], isum)

        return ii

    def __rel_update(self):
        """Update lists with component relationships"""
        self._parents = self.__get_parents()
        self._childs = self.__get_childs()
        self._topo_nodes = self.__get_topo_sort()

    def __get_parent_name(self, node):
        """Get parent name of node"""
        if self._parents[node] == -1:
            return ""

        return self._g[self._parents[node][0]].params["name"]

    def solve(self, *, vtol=1e-5, itol=1e-6, maxiter=1000, quiet=True):
        """Analyze system"""
        self.__rel_update()
        # initial condition
        v, i = self.__sys_init()
        # solve system function
        iters = 1
        while iters <= maxiter:
            vi = self.__fwd_prop(v, i)
            ii = self.__back_prop(vi, i)
            if np.allclose(np.array(v), np.array(vi), rtol=vtol) and np.allclose(
                np.array(i), np.array(ii), rtol=itol
            ):
                if not quiet:
                    print("Tolerances met after {} iterations".format(iters))
                break
            v, i = vi, ii
            iters += 1

        if iters > maxiter:
            print("Analysis aborted after {} iterations".format(iters - 1))
            return None

        # calculate results for each node
        names, parent, typ, pwr, loss = [], [], [], [], []
        eff, warn, vsi, iso, vso, isi = [], [], [], [], [], []
        domain, dname = [], "none"
        sources, dwarns = {}, {}
        for n in self._topo_nodes:  # [vi, vo, ii, io]
            names += [self._g[n].params["name"]]
            if self._g[n].component_type.name == "SOURCE":
                dname = self._g[n].params["name"]
            domain += [dname]
            vi = v[n]
            vo = v[n]
            ii = i[n]
            io = i[n]
            p = self._parents[n]

            if p == -1:  # root
                vi = v[n] + self._g[n].params["rs"] * ii
            elif self._childs[n] == -1:  # leaf
                vi = v[p[0]]
                io = 0.0
            else:
                io = 0.0
                for c in self._childs[n]:
                    io += i[c]
                vi = v[p[0]]
            parent += [self.__get_parent_name(n)]
            p, l, e = self._g[n]._solv_pwr_loss(vi, vo, ii, io)
            pwr += [p]
            loss += [l]
            eff += [e]
            typ += [self._g[n].component_type.name]
            if self._g[n].component_type.name == "SOURCE":
                sources[dname] = vi
                dwarns[dname] = 0
            w = self._g[n]._solv_get_warns(vi, vo, ii, io)
            warn += [w]
            if w != "":
                dwarns[dname] = 1
            vsi += [vi]
            iso += [io]
            vso += [v[n]]
            isi += [i[n]]

        # subsystems summary
        for d in range(len(sources)):
            names += ["Subsystem {}".format(list(sources.keys())[d])]
            typ += [""]
            parent += [""]
            domain += [""]
            vsi += [sources[list(sources.keys())[d]]]
            vso += [""]
            isi += [""]
            iso += [""]
            pwr += [""]
            loss += [""]
            eff += [""]
            if dwarns[list(sources.keys())[d]] > 0:
                warn += ["Yes"]
            else:
                warn += [""]

        # system total
        names += ["System total"]
        typ += [""]
        parent += [""]
        domain += [""]
        vsi += [""]
        vso += [""]
        isi += [""]
        iso += [""]
        pwr += [""]
        loss += [""]
        eff += [""]
        if any(warn):
            warn += ["Yes"]
        else:
            warn += [""]

        # report
        res = {}
        res["Component"] = names
        res["Type"] = typ
        res["Parent"] = parent
        res["Domain"] = domain
        res["Vin (V)"] = vsi
        res["Vout (V)"] = vso
        res["Iin (A)"] = isi
        res["Iout (A)"] = iso
        res["Power (W)"] = pwr
        res["Loss (W)"] = loss
        res["Efficiency (%)"] = eff
        res["Warnings"] = warn
        df = pd.DataFrame(res)

        # update subsystem power/loss/efficiency
        for d in range(len(sources)):
            src = list(sources.keys())[d]
            idx = df[df.Component == "Subsystem {}".format(src)].index[0]
            pwr = df[(df.Domain == src) & (df.Type == "SOURCE")]["Power (W)"].values[0]
            df.at[idx, "Power (W)"] = pwr
            loss = df[df.Domain == src]["Loss (W)"].sum()
            df.at[idx, "Loss (W)"] = loss
            df.at[idx, "Efficiency (%)"] = _get_eff(pwr, loss)

        # update system total
        pwr = df[(df.Domain == "") & (df["Power (W)"] != "")]["Power (W)"].sum()
        idx = df.index[-1]
        df.at[idx, "Power (W)"] = pwr
        loss = df[(df.Domain == "") & (df["Loss (W)"] != "")]["Loss (W)"].sum()
        df.at[idx, "Loss (W)"] = loss
        df.at[idx, "Efficiency (%)"] = _get_eff(pwr, loss)

        # if only one subsystem, delete subsystem row
        if len(sources) < 2:
            df.drop(len(df) - 2, inplace=True)
            df.reset_index(inplace=True, drop=True)
        return df

    def params(self, limits=False):
        """Return component parameters"""
        self.__rel_update()
        names, typ, parent, vo, vdrop = [], [], [], [], []
        iq, rs, eff, ii, pwr = [], [], [], [], []
        lii, lio, lvi, lvo = [], [], [], []
        domain, dname = [], "none"
        for n in self._topo_nodes:
            names += [self._g[n].params["name"]]
            typ += [self._g[n].component_type.name]
            if self._g[n].component_type.name == "SOURCE":
                dname = self._g[n].params["name"]
            domain += [dname]
            _vo, _vdrop, _iq, _rs, _eff, _ii, _pwr = "", "", "", "", "", "", ""
            if self._g[n].component_type == ComponentTypes.SOURCE:
                _vo = self._g[n].params["vo"]
                _rs = self._g[n].params["rs"]
            elif self._g[n].component_type == ComponentTypes.LOAD:
                if "pwr" in self._g[n].params:
                    _pwr = self._g[n].params["pwr"]
                elif "rs" in self._g[n].params:
                    _rs = self._g[n].params["rs"]
                else:
                    _ii = self._g[n].params["ii"]
            elif self._g[n].component_type == ComponentTypes.CONVERTER:
                _vo = self._g[n].params["vo"]
                _iq = self._g[n].params["iq"]
                _eff = self._g[n].params["eff"]
            elif self._g[n].component_type == ComponentTypes.LINREG:
                _vo = self._g[n].params["vo"]
                _vdrop = self._g[n].params["vdrop"]
                _iq = self._g[n].params["iq"]
            elif self._g[n].component_type == ComponentTypes.LOSS:
                _vdrop = self._g[n].params["vdrop"]
                _rs = self._g[n].params["rs"]
            vo += [_vo]
            vdrop += [_vdrop]
            iq += [_iq]
            rs += [_rs]
            eff += [_eff]
            ii += [_ii]
            pwr += [_pwr]
            parent += [self.__get_parent_name(n)]
            if limits:
                lii += [_get_opt(self._g[n].limits, "ii", LIMITS_DEFAULT["ii"])]
                lio += [_get_opt(self._g[n].limits, "io", LIMITS_DEFAULT["io"])]
                lvi += [_get_opt(self._g[n].limits, "vi", LIMITS_DEFAULT["vi"])]
                lvo += [_get_opt(self._g[n].limits, "vo", LIMITS_DEFAULT["vo"])]
        # report
        res = {}
        res["Component"] = names
        res["Type"] = typ
        res["Parent"] = parent
        res["Domain"] = domain
        res["vo (V)"] = vo
        res["vdrop (V)"] = vdrop
        res["iq (A)"] = iq
        res["rs (Ohm)"] = rs
        res["eff (%)"] = eff
        res["ii (A)"] = ii
        res["pwr (W)"] = pwr
        if limits:
            res["vi limits (V)"] = lvi
            res["vo limits (V)"] = lvo
            res["ii limits (A)"] = lii
            res["io limits (A)"] = lio
        return pd.DataFrame(res)

    def save(self, fname, *, indent=4):
        """Save system as json file"""
        self.__rel_update()
        sys = {
            "name": self._g.attrs["name"],
        }
        ridx = self.__get_sources()
        root = [self._g[n].params["name"] for n in ridx]
        for r in range(len(ridx)):
            tree = self.__get_childs_tree(ridx[r])
            cdict = {}
            if tree != {}:
                for e in tree:
                    childs = []
                    for c in tree[e]:
                        childs += [
                            {
                                "type": self._g[c].component_type.name,
                                "params": self._g[c].params,
                                "limits": self._g[c].limits,
                            }
                        ]
                    cdict[self._g[e].params["name"]] = childs
            sys[root[r]] = {
                "type": self._g[ridx[r]].component_type.name,
                "params": self._g[ridx[r]].params,
                "limits": self._g[ridx[r]].limits,
                "childs": cdict,
            }

        with open(fname, "w") as f:
            json.dump(sys, f, indent=indent)

![case1](case_1.png)

In [19]:
case13 = System("Case13 system", Source("3.3V", vo=3.3))
case13.add_source(Source("12V", vo=12, limits={"io":[0, 1e-3]}))
case13.add_comp("3.3V", comp=PLoad("MCU", pwr=0.2))
case13.add_comp("12V", comp=PLoad("Test", pwr=1.5))
case13.add_source(Source("3.3V aux", vo=3.3, limits={"io":[0, 20e-3]}))
case13.tree()

Case13 system
├── 3.3V aux
├── 12V
│   └── Test
└── 3.3V
    └── MCU

In [21]:
dir(case13)

['_System__back_prop',
 '_System__chk_name',
 '_System__chk_parent',
 '_System__fwd_prop',
 '_System__get_childs',
 '_System__get_childs_tree',
 '_System__get_index',
 '_System__get_nodes',
 '_System__get_parent_name',
 '_System__get_parents',
 '_System__get_sources',
 '_System__get_topo_sort',
 '_System__make_rtree',
 '_System__rel_update',
 '_System__sys_init',
 '_System__sys_vars',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_childs',
 '_g',
 '_parents',
 '_topo_nodes',
 'add_comp',
 'add_source',
 'change_comp',
 'del_comp',
 'from_file',
 'params',
 'save',
 'solve',
 'tree']

In [15]:
case13.params(limits=True)

,Component,Type,Parent,Domain,vo (V),vdrop (V),iq (A),rs (Ohm),eff (%),ii (A),pwr (W),vi limits (V),vo limits (V),ii limits (A),io limits (A)
0,3.3V aux,SOURCE,,3.3V aux,3.3,,,0.0,,,,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0, 0.02]"
1,12V,SOURCE,,12V,12,,,0.0,,,,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0, 0.001]"
2,Test,LOAD,12V,12V,,,,,,,1.5,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]"
3,3.3V,SOURCE,,3.3V,3.3,,,0.0,,,,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]"
4,MCU,LOAD,3.3V,3.3V,,,,,,,0.2,"[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]","[0.0, 1000000.0]"


In [17]:
case13._g.attrs

{'name': 'Case13 system',
 'nodes': {'3.3V': 0, '12V': 1, 'MCU': 2, 'Test': 3, '3.3V aux': 4}}

In [7]:
case1 = System('Bluetooth sensor', Source('3V coin', vo=3, rs=13e-3))
case1.add_comp('3V coin', comp=Converter('1.8V buck', vo=1.8, eff=0.87, iq=12e-6))
#case1.add_element('1.8V buck', element=Loss('Resistor2', rs=5.0))
case1.add_comp('1.8V buck', comp=PLoad('MCU', pwr=27e-3))
case1.add_comp('3V coin', comp=Converter('5V boost', vo=5, eff=0.91, iq=42e-6))
case1.add_comp('5V boost', comp=ILoad('Sensor', ii=15e-3))
case1.add_comp(parent='5V boost', comp=Loss('RC filter', rs=33.0))
case1.add_comp('RC filter', comp=LinReg('LDO 2.5V', vo=2.5, vdrop=0.27, iq=150e-6))
case1.add_comp('LDO 2.5V', comp=PLoad('ADC', pwr=15e-3))
case1.add_comp('5V boost', comp=RLoad('Res divider', rs=200e3))
case1.add_source(Source('12V', vo=12, rs=22e-3))
case1.add_comp('12V', comp=PLoad('Fan', pwr=0.1))
#case1.g.attrs
case1.tree()

Bluetooth sensor
├── 12V
│   └── Fan
└── 3V coin
    ├── 5V boost
    │   ├── Res divider
    │   ├── RC filter
    │   │   └── LDO 2.5V
    │   │       └── ADC
    │   └── Sensor
    └── 1.8V buck
        └── MCU

In [8]:
df = case1.solve(quiet=False)
#case1.save("new_format.json")
df

Tolerances met after 8 iterations


,Component,Type,Parent,Domain,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,12V,SOURCE,,12V,12.0,11.999817,0.008333,0.008333,0.1,0.000002,99.998472,
1,Fan,LOAD,12V,12V,11.999817,0.0,0.008333,0.0,0.1,0.0,100.0,
2,3V coin,SOURCE,,3V coin,3.0,2.999361,0.049191,0.049191,0.147543,0.000031,99.978679,
3,5V boost,CONVERTER,3V coin,3V coin,2.999361,5,0.038832,0.021175,0.0,0.010597,90.901577,
4,Res divider,LOAD,5V boost,3V coin,5,0.0,0.000025,0.0,0.000125,0.0,100.0,
5,RC filter,LOSS,5V boost,3V coin,5,4.79705,0.00615,0.00615,0.0,0.001248,95.941,
6,LDO 2.5V,LINREG,RC filter,3V coin,4.79705,2.5,0.00615,0.006,0.0,0.014502,50.844256,
7,ADC,LOAD,LDO 2.5V,3V coin,2.5,0.0,0.006,0.0,0.015,0.0,100.0,
8,Sensor,LOAD,5V boost,3V coin,5,0.0,0.015,0.0,0.075,0.0,100.0,
9,1.8V buck,CONVERTER,3V coin,3V coin,2.999361,1.8,0.010359,0.015,0.0,0.00407,86.899218,


In [87]:
df

,Component,Type,Parent,Domain,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3V coin,3.0,2.999361,0.049191,0.049191,0.147543,0.000031,0.999787,
1,5V boost,CONVERTER,3V coin,3V coin,2.999361,5,0.038832,0.021175,0.0,0.010597,0.909016,
2,Res divider,LOAD,5V boost,3V coin,5,0.0,0.000025,0.0,0.000125,0.0,100.0,
3,RC filter,LOSS,5V boost,3V coin,5,4.79705,0.00615,0.00615,0.0,0.001248,0.95941,
4,LDO 2.5V,LINREG,RC filter,3V coin,4.79705,2.5,0.00615,0.006,0.0,0.014502,0.508443,
5,ADC,LOAD,LDO 2.5V,3V coin,2.5,0.0,0.006,0.0,0.015,0.0,100.0,
6,Sensor,LOAD,5V boost,3V coin,5,0.0,0.015,0.0,0.075,0.0,100.0,
7,1.8V buck,CONVERTER,3V coin,3V coin,2.999361,1.8,0.010359,0.015,0.0,0.00407,0.868992,
8,MCU,LOAD,1.8V buck,3V coin,1.8,0.0,0.015,0.0,0.027,0.0,100.0,
10,System total,,,,,,,,0.147543,0.030449,0.793625,


In [69]:
case1b = System.from_file("new_format.json")

In [70]:
case1b.solve()

,Component,Type,Parent,Domain,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3V coin,3.0,2.999361,0.049191,0.049191,0.147543,0.000031,0.999787,
1,1.8V buck,CONVERTER,3V coin,3V coin,2.999361,1.8,0.010359,0.015,0.0,0.00407,0.868992,
2,MCU,LOAD,1.8V buck,3V coin,1.8,0.0,0.015,0.0,0.027,0.0,100.0,
3,5V boost,CONVERTER,3V coin,3V coin,2.999361,5,0.038832,0.021175,0.0,0.010597,0.909016,
4,Sensor,LOAD,5V boost,3V coin,5,0.0,0.015,0.0,0.075,0.0,100.0,
5,RC filter,LOSS,5V boost,3V coin,5,4.79705,0.00615,0.00615,0.0,0.001248,0.95941,
6,LDO 2.5V,LINREG,RC filter,3V coin,4.79705,2.5,0.00615,0.006,0.0,0.014502,0.508443,
7,ADC,LOAD,LDO 2.5V,3V coin,2.5,0.0,0.006,0.0,0.015,0.0,100.0,
8,Res divider,LOAD,5V boost,3V coin,5,0.0,0.000025,0.0,0.000125,0.0,100.0,
9,12V,SOURCE,,12V,12.0,11.999817,0.008333,0.008333,0.1,0.000002,0.999985,


In [187]:
#case1.save('test2.json')
case1.params()

,Component,Type,Parent,Domain,vo (V),vdrop (V),iq (A),rs (Ohm),eff (%),ii (A),pwr (W)
0,12V,SOURCE,,12V,12,,,0.022,,,
1,Fan,LOAD,12V,12V,,,,,,,0.1
2,3V coin,SOURCE,,3V coin,3,,,0.013,,,
3,5V boost,CONVERTER,3V coin,3V coin,5,,0.000042,,0.91,,
4,Res divider,LOAD,5V boost,3V coin,,,,200000.0,,,
5,RC filter,LOSS,5V boost,3V coin,,0.0,,33.0,,,
6,LDO 2.5V,LINREG,RC filter,3V coin,2.5,0.27,0.00015,,,,
7,ADC,LOAD,LDO 2.5V,3V coin,,,,,,,0.015
8,Sensor,LOAD,5V boost,3V coin,,,,,,0.015,
9,1.8V buck,CONVERTER,3V coin,3V coin,1.8,,0.000012,,0.87,,


In [177]:
%timeit case1.solve()

4.07 ms ± 12 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
s = case1.save("new_format.json")

9 12V
0 3V coin


In [30]:
for p in list(s["12V"]["childs"].keys()):
    print(p)
    

12V


In [31]:
s["12V"]["childs"]["12V"]

[{'type': 'LOAD',
  'params': {'name': 'Fan', 'pwr': 0.1},
  'limits': {'vi': [0.0, 1000000.0],
   'vo': [0.0, 1000000.0],
   'ii': [0.0, 1000000.0],
   'io': [0.0, 1000000.0]}}]

In [14]:
li = list(s.keys())
for e in range(1, len(li)):
    print(s[li[e]]["childs"])
    for p in s[li[e]]["childs"]:
        print(p)
        for c in s[li[e]][p]:
            print(c)

{'12V': [{'type': 'LOAD', 'params': {'name': 'Fan', 'pwr': 0.1}, 'limits': {'vi': [0.0, 1000000.0], 'vo': [0.0, 1000000.0], 'ii': [0.0, 1000000.0], 'io': [0.0, 1000000.0]}}]}
12V


KeyError: '12V'

In [6]:
#case1.change_element(name='MCU', element=Load('Big MCU', pwr=130e-3))
#case1.change_element(name='MCU', element=Loss('Resistor1', res=133.0))
#case1.change_element(name='1.8V buck', element=Loss('Resistor2', res=7.8))
#case1.add_element('ADC', element='dummy')
case1.del_comp(name='5V boost', del_childs=False)
case1.tree()

3V coin
├── Sensor
├── RC filter
│   └── LDO 2.5V
│       └── ADC
├── Res divider
└── 1.8V buck
    └── MCU

In [7]:
case1.solve()

,Component,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.00000,2.99959,0.031523,0.031523,0.094557,0.000013,0.999863,
1,Sensor,LOAD,3V coin,2.99959,1.80000,0.010358,0.000000,0.044994,0.000000,100.000000,
2,RC filter,LOSS,3V coin,2.99959,0.00000,0.015000,0.006150,0.000000,0.001248,0.932341,
3,LDO 2.5V,LINREG,RC filter,2.79664,0.00000,0.015000,0.006000,0.000000,0.002199,0.872127,
4,ADC,LOAD,LDO 2.5V,2.50000,2.79664,0.006150,0.000000,0.015000,0.000000,100.000000,
5,Res divider,LOAD,3V coin,2.99959,2.50000,0.006150,0.000000,0.000045,0.000000,100.000000,
6,1.8V buck,CONVERTER,3V coin,2.99959,0.00000,0.006000,0.015000,0.000000,0.004070,0.868992,
7,MCU,LOAD,1.8V buck,1.80000,0.00000,0.000015,0.000000,0.027000,0.000000,100.000000,
8,System total,,,3.00000,0.00000,0.031523,0.000000,0.094570,0.007531,0.920367,None


| Element | vo (V)   | vdrop (V) | iq (A) | rs (Ohm) | eff (%) | ii (A) | pwr (W) | is (A) |
|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|
|  **Source**  | # |     |   | (#) |  |  |  | |
|  **ILoad**   |     |     || || #||(#) |
|  **PLoad**   |     |    ||||| #|(#) |
|  **RLoad**   |     |    ||#||| |(#) |
|  **Converter**   |  #   |     | (#)||#|||(#) |
|  **LinReg**  |  # | (#)   | (#) |||||(#) |
|  **Loss**  |     | #   || # |||| |

In [128]:
case2 = System('Case2', Source('3V coin', vo=3, rs=13e-3))
#case2.add_element('3V coin', element=Loss('RC filter', rs=33.0))
#case2.solve(quiet=False)
case2.save('test3.json')
case2.solve()

,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.0,3.0,0.0,0.0,0.0,0.0,100.0,
1,System total,,,3.0,0.0,0.0,0.0,0.0,0.0,0.0,None


In [60]:
sys = {'name': 'system name', 'root':
       {'type': 'SOURCE', 'params': {'name': '3V coin', 'vo': 3, 'rs': 0.013}, 
        'limits':{'vi': [0.0, 1000000.0], 'vo': [0.0, 1000000.0], 'ii': [0.0, 1000000.0], 'io': [0.0, 1000000.0]}}}
sys

{'name': 'system name',
 'root': {'type': 'SOURCE',
  'params': {'name': '3V coin', 'vo': 3, 'rs': 0.013},
  'limits': {'vi': [0.0, 1000000.0],
   'vo': [0.0, 1000000.0],
   'ii': [0.0, 1000000.0],
   'io': [0.0, 1000000.0]}}}

In [30]:
root = case1.g[0].params['name']
adj = rx.bfs_successors(case1.g, case1.g.attrs[root])
ndict = {}
for i in adj:
    c = []
    for j in i[1]:
        c += [j.params['name']]
    ndict[i[0].params['name']] = c
ndict

{'3V coin': ['5V boost', '1.8V buck'],
 '5V boost': ['RC filter', 'Sensor'],
 '1.8V buck': ['MCU'],
 'RC filter': ['LDO 2.5V'],
 'LDO 2.5V': ['ADC']}

In [236]:
c3 = System.from_file('test3.json')
c3.solve(quiet=False)

Tolerances met after 1 iterations


,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.0,3.0,0.0,0.0,0.0,0.0,100.0,
1,System total,,,3.0,0.0,0.0,0.0,0.0,0.0,0.0,None


In [188]:
c3.params()

,Element,Type,Parent,vo (V),vdrop (V),iq (A),rs (Ohm),eff (%),ii (A),pwr (W)
0,3V coin,SOURCE,,3,,,0.013,,,


In [58]:
tree = {0: [3, 1], 3: [5, 4], 1: [2], 5: [6], 6: [7]}
cdict = {}
for e in tree:
    print(case1.g[e].params['name'], tree[e])
    childs = []
    for c in tree[e]:
        childs += [{'type': case1.g[c].element_type.name, 'params':case1.g[c].params, 'limits':case1.g[c].limits}]
    cdict[case1.g[e].params['name']] = childs

3V coin [3, 1]
5V boost [5, 4]
1.8V buck [2]
RC filter [6]
LDO 2.5V [7]


In [261]:
case2 = System('Bluetooth sensor', Source('3V coin', vo=3, rs=13e-3))
#case2.save('test3.json')
case2.solve()

,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.0,3.0,0.0,0.0,0.0,0.0,100.0,
1,System total,,,3.0,0.0,0.0,0.0,0.0,0.0,0.0,None


In [66]:
case1.g.attrs

{'3V coin': 0,
 '1.8V buck': 1,
 'MCU': 2,
 '5V boost': 3,
 'Sensor': 4,
 'RC filter': 5,
 'LDO 2.5V': 6,
 'ADC': 7}

In [10]:
case1b = System.from_file('test2.json')
#case3 = system_from_file('test2.json')
case1b.g.attrs

3V coin
  5V boost
  1.8V buck
5V boost
  RC filter
  Sensor
1.8V buck
  MCU
RC filter
  LDO 2.5V
LDO 2.5V
  ADC


{'name': 'Bluetooth sensor',
 'nodes': {'3V coin': 0,
  '5V boost': 1,
  '1.8V buck': 2,
  'RC filter': 3,
  'Sensor': 4,
  'MCU': 5,
  'LDO 2.5V': 6,
  'ADC': 7}}

In [11]:
case1b.solve(quiet=False)

Tolerances met after 8 iterations


,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.000000,2.999361,0.049146,0.049146,0.147405,0.000031,0.999787,
1,5V boost,CONVERTER,3V coin,2.999361,5.000000,0.038787,0.021150,0.000000,0.010585,0.909015,
2,1.8V buck,CONVERTER,3V coin,2.999361,1.800000,0.010359,0.015000,0.000000,0.004070,0.868992,
3,RC filter,LOSS,5V boost,5.000000,4.797050,0.006150,0.006150,0.000000,0.001248,0.959410,
4,Sensor,LOAD,5V boost,5.000000,0.000000,0.015000,0.000000,0.075000,0.000000,100.000000,
5,MCU,LOAD,1.8V buck,1.800000,0.000000,0.015000,0.000000,0.027000,0.000000,100.000000,
6,LDO 2.5V,LINREG,RC filter,4.797050,2.500000,0.006150,0.006000,0.000000,0.014502,0.508443,
7,ADC,LOAD,LDO 2.5V,2.500000,0.000000,0.006000,0.000000,0.015000,0.000000,100.000000,
8,System total,,,3.000000,0.000000,0.049146,0.000000,0.147437,0.030437,0.793561,None


In [240]:
case3.params()

,Element,Type,Parent,vo (V),vdrop (V),iq (A),rs (Ohm),eff (%),ii (A),pwr (W)
0,3V coin,SOURCE,,3,,,0.013,,,
1,5V boost,CONVERTER,3V coin,5,,0.000042,,0.91,,
2,1.8V buck,CONVERTER,3V coin,1.8,,0.000012,,0.87,,
3,RC filter,LOSS,5V boost,,0.0,,33.0,,,
4,Sensor,LOAD,5V boost,,,,,,0.015,
5,MCU,LOAD,1.8V buck,,,,,,,0.027
6,LDO 2.5V,LINREG,RC filter,2.5,0.27,0.00015,,,,
7,ADC,LOAD,LDO 2.5V,,,,,,,0.015


In [242]:
case1.solve()

,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.000000,2.999718,0.021665,0.021665,0.064988,0.000006,0.999906,
1,1.8V buck,CONVERTER,3V coin,2.999718,1.800000,0.010358,0.015000,0.000000,0.004070,0.868992,
2,MCU,LOAD,1.8V buck,1.800000,0.000000,0.015000,0.000000,0.027000,0.000000,100.000000,
3,5V boost,CONVERTER,3V coin,2.999718,5.000000,0.011307,0.021150,0.000000,0.003167,0.906620,
4,Sensor,LOAD,5V boost,5.000000,0.000000,0.015000,0.000000,0.075000,0.000000,100.000000,
5,RC filter,LOSS,5V boost,5.000000,4.797050,0.006150,0.006150,0.000000,0.001248,0.959410,
6,LDO 2.5V,LINREG,RC filter,4.797050,2.500000,0.006150,0.006000,0.000000,0.014502,0.508443,
7,ADC,LOAD,LDO 2.5V,2.500000,0.000000,0.006000,0.000000,0.015000,0.000000,100.000000,
8,System total,,,3.000000,0.000000,0.021665,0.000000,0.064994,0.022994,0.646216,None


In [243]:
case1.params()

,Element,Type,Parent,vo (V),vdrop (V),iq (A),rs (Ohm),eff (%),ii (A),pwr (W)
0,3V coin,SOURCE,,3,,,0.013,,,
1,1.8V buck,CONVERTER,3V coin,1.8,,0.000012,,0.87,,
2,MCU,LOAD,1.8V buck,,,,,,,0.027
3,5V boost,CONVERTER,3V coin,5,,0.000042,,0.91,,
4,Sensor,LOAD,5V boost,,,,,,0.015,
5,RC filter,LOSS,5V boost,,0.0,,33.0,,,
6,LDO 2.5V,LINREG,RC filter,2.5,0.27,0.00015,,,,
7,ADC,LOAD,LDO 2.5V,,,,,,,0.015


In [244]:
case3.save('test2b.json')

In [245]:
case4 = System.from_file('test2b.json')
case4.solve()

3V coin
  1.8V buck
  5V boost
1.8V buck
  MCU
5V boost
  Sensor
  RC filter
RC filter
  LDO 2.5V
LDO 2.5V
  ADC


,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.000000,2.999718,0.021665,0.021665,0.064988,0.000006,0.999906,
1,1.8V buck,CONVERTER,3V coin,2.999718,1.800000,0.010358,0.015000,0.000000,0.004070,0.868992,
2,5V boost,CONVERTER,3V coin,2.999718,5.000000,0.011307,0.021150,0.000000,0.003167,0.906620,
3,MCU,LOAD,1.8V buck,1.800000,0.000000,0.015000,0.000000,0.027000,0.000000,100.000000,
4,Sensor,LOAD,5V boost,5.000000,0.000000,0.015000,0.000000,0.075000,0.000000,100.000000,
5,RC filter,LOSS,5V boost,5.000000,4.797050,0.006150,0.006150,0.000000,0.001248,0.959410,
6,LDO 2.5V,LINREG,RC filter,4.797050,2.500000,0.006150,0.006000,0.000000,0.014502,0.508443,
7,ADC,LOAD,LDO 2.5V,2.500000,0.000000,0.006000,0.000000,0.015000,0.000000,100.000000,
8,System total,,,3.000000,0.000000,0.021665,0.000000,0.064994,0.022994,0.646216,None


In [246]:
df =case1.solve()

In [64]:
tn = [n for n in rx.topological_sort(case1.g)]
[n for n in tn if isinstance(case1.g[n], Source)]

[9, 0]

In [18]:
ridx = [9, 0]
[case1.g[n].params["name"] for n in ridx]

['12V', '3V coin']

In [65]:
tn

[9, 10, 0, 3, 8, 5, 6, 7, 4, 1, 2]

In [74]:
[n for n in case1.g.node_indices()]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]